# Read Sent Headers
## 2022-01-19
- Read all emails from a folder and identify all lines with the "Sent:" header. 
- Record the filename, and the line number within the filename, for analysis

In [13]:
%load_ext autoreload
%autoreload 2

In [10]:
import numpy as np
import pandas as pd
import os, re
import sys
from datetime import datetime
from glob import glob
from tqdm.notebook import tqdm, tnrange
import function_library3 as fctlib

In [4]:
# inputpath="./email_txt/downloaded_from_data_tallahassee/"
inputpath="./test_output/"
# inputpath="./23-11/"
# inputpath="./34-3/"
# inputpath="./18-4/"
inputpath="./10-8/"
files = glob(inputpath + "*.txt")  # subset of files
# files = [inputpath + "test2.txt"]
len(files)

9

In [5]:
files_dict = {}
lines = []
count = 0

for file in tqdm(files):
    with open(file, "r") as f:
        local_lines = []
        files_dict[file] = f.read().splitlines(); # \n removed
        append_line = False
        # cmdline = "grep -i isautomessage %s" % file
        # os.system(cmdline)
        for line_no, line in enumerate(files_dict[file]):
            if (re.search("Sent:", line)): 
                full_line = f"{line_no}, {file},  ===> , {line}"
                append_line = True
                local_lines.append(full_line)
                continue
            if (re.match("isAutoMessage:", line)):
                append_line = False
                count += 1
                break
            if (re.match("isDisplacement:", line)):
                append_line = False
                count += 1
                break

        if append_line:
            lines.extend(local_lines)
            
# All emails with isAutoMessage=True are ignored
print("count: ", count)
print(len(lines))


  0%|          | 0/9 [00:00<?, ?it/s]

count:  1
25


In [6]:
for line in lines[0:10]:
    print(line)

3, ./10-8/test2.txt,  ===> , Sent: Wednesday, May 18, 2016 10:35 AM
477, ./10-8/test2.txt,  ===> , Sent:
1, ./10-8/test3.txt,  ===> , Sent: Thursday, June 01, 2017 3:27 PM
518, ./10-8/test3.txt,  ===> , Sent: Friday, May 20, 2016 10:28 AM
557, ./10-8/test3.txt,  ===> , Sent: Friday, May 20, 2016 10:28 AM
596, ./10-8/test3.txt,  ===> , Sent: Friday, May 20, 2016 10:28 AM
703, ./10-8/test3.txt,  ===> , Sent: Wednesday, May 18, 2016 10:35 AM
737, ./10-8/test3.txt,  ===> , Sent: Wednesday, May 18, 2016 10:35 AM
771, ./10-8/test3.txt,  ===> , Sent: Wednesday, May 18, 2016 10:35 AM
1245, ./10-8/test3.txt,  ===> , Sent:


### Save Sent: Headers to a file

In [7]:
print("before, len(lines) ", len(lines))

before, len(lines)  25


In [8]:
len(lines)

25

In [16]:
filters = fctlib.createDateFilters()
subject_spelling = re.compile(r"^.*(S\s+?u\s+?j\s+?e\s+?c\s+?t\s+?)")

In [119]:
def remove_file_id(line):
    return re.sub('[0-9]+-[0-9]+$', '', line) 

In [120]:
### Only keep doubtful dates
print(len(lines))
# filters.append(re.compile('AM$'))
kept = []
for line in lines:
    keep = True
    line = remove_file_id(line)
    line = subject_spelling.sub("Subject", line)
    for regex, value in filters.items():
        if regex.match(line):
            keep = False
    if keep: 
        kept.append(line)
print(len(kept))
kept1 = kept

21
10


In [121]:
# order kept array according to mail index
ids = []
id_search = re.compile(r"^.*?([0-9]+)_fn.*$")
for row in kept:
    # new_row = id_search.search(r"\1", row)
    # idd = re.sub(r"([0-9]+?)_fn", r"\1", row)
    # print("row: ...", row)
    match = re.match(r"^.*?(\d+)_fn.*$", row)
    # print(match)
    if match:
        idd = match.group(1)
        ids.append(int(idd))
        
ids = np.array(ids)
args = np.argsort(ids)
print(len(ids))
print(ids[args][0:20])
kept1 = np.array(kept)[args]

1
[1]


In [122]:
np.savetxt("Sent_headers.txt", kept1, fmt="%s")

## Sent Headers saved to file
---

In [52]:
files_dict[files[0]]

['To: File Cascades Holdings, LLC dba The Edison Restaurant',
 'From: Ruben Alfaras, Senior Vice President',
 'Date: 5/ 1/2015',
 'Subject: Assignment of Lease/Subordination Agreement',
 '',
 '470 Suwannee Street',
 'Tallahassee, FL 32301',
 '',
 ' ',
 '',
 'City of?l?allahassee is the landlord for the above referenced property. When Lender?s',
 'counsel requested the Assignment of Lease/Subordination Agreement, City would not',
 'agree to Sign unless we would accept the ?Except for? items noted on the collateral',
 'description. Since the City is spending a great deal on this project and our funds will',
 'mostly be used for equipment that can be removed, we have agreed to this request.',
 '',
 'The Bank. feels this is a risk worth taking and would do the some were this a conventional',
 'bank loan.',
 '',
 'W,']

In [53]:
# for k in files_dict.keys(): print(k)

In [54]:
for k in files_dict.keys():
    # print(len(files_dict[k]))
    pass

In [55]:
prefixes = ['date', 'sent', 'to', 'cc', 're', 'bcc', 'when', 'subject', 'original', 'from']
folder = "search_results"
# prefixes = ['date']

In [56]:
breakup_date = re.compile(r'([\w-]+?.txt):(\d+?):(.*)')

In [57]:
dicts = {}
for prefix in prefixes:
    inputpath = os.path.join(folder, prefix + "_txt")
    folder_list = []
    dicts[prefix] = folder_list

    with open(inputpath, "r", encoding="utf8") as f:
        print("inputpath: ", inputpath)
        for line in f.readlines():
            filenm, ln, rest = breakup_date.findall(line)[0]
            # grep -n numbers lines from 1, and I want 0-based
            d = {"filenm": filenm, "ln": int(ln)-1, "rest": rest.rstrip()}
            folder_list.append(d)

inputpath:  search_results/date_txt
inputpath:  search_results/sent_txt
inputpath:  search_results/to_txt
inputpath:  search_results/cc_txt
inputpath:  search_results/re_txt
inputpath:  search_results/bcc_txt
inputpath:  search_results/when_txt
inputpath:  search_results/subject_txt
inputpath:  search_results/original_txt
inputpath:  search_results/from_txt


In [58]:
dfs = {}
for k in dicts.keys():
    dfs[k] = pd.DataFrame(dicts[k])
    print(k, dfs[k].shape)

date (15465, 3)
sent (58640, 3)
to (71105, 3)
cc (36448, 3)
re (33718, 3)
bcc (172, 3)
when (647, 3)
subject (74181, 3)
original (1, 3)
from (71557, 3)


In [59]:
dates = dfs['date']
dates.shape

(15465, 3)

In [60]:
dfs['date'].head()

,filenm,ln,rest
0,10-1-Cascade-2013.txt,133,"Date: Thursday, July 11, 2013 1:08 PM"
1,10-1-Cascade-2013.txt,1381,"Date: August 20, 2013, 6:24:26 PM EDT"
2,10-1-Cascade-2013.txt,1722,"Date: Tue, Jun 25, 2013 at 11:45 AM"
3,10-1-Cascade-2013.txt,2106,"Date: Tue, Jun 25, 2013 at 11:45 AM"
4,10-1-Cascade-2013.txt,2127,"Date: Tue, Jun 25, 2013 at 11:45 AM"


In [61]:
v = dates['rest'].values
len(v)

15465

In [62]:
rest = dfs['date'].loc[:, 'rest'].values
rest = [r.strip() for r in rest]
rest = [r[5:].strip() if r[0:5].lower() == 'date:' else r.strip() for r in rest]
for i, r in enumerate(rest):
    rest[i] = re.sub("\(GMT-05:00\)", "", r)
dfs['date']['rest'] = rest
len(rest)

15465

In [63]:
dfs.keys()

dict_keys(['date', 'sent', 'to', 'cc', 're', 'bcc', 'when', 'subject', 'original', 'from'])

In [64]:
rest = dfs['sent'].loc[:, 'rest'].values
rest = [r.strip() for r in rest]
rest = [r[5:].strip() if r[0:5].lower() == 'sent:' else r.strip() for r in rest]
for i, r in enumerate(rest):
    rr = re.sub("\(GMT-05:00\).*", "", r)
    rr = re.sub("\(UTC-05:00\).*", "", rr)
    rest[i] = rr
dfs['sent']['rest'] = rest
len(rest)

58640

In [65]:
dfs['date'].shape, len(rest)

((15465, 3), 58640)

In [40]:
dfs['date'].head()

,filenm,ln,rest
0,10-1-Cascade-2013.txt,133,"Thursday, July 11, 2013 1:08 PM"
1,10-1-Cascade-2013.txt,1381,"August 20, 2013, 6:24:26 PM EDT"
2,10-1-Cascade-2013.txt,1722,"Tue, Jun 25, 2013 at 11:45 AM"
3,10-1-Cascade-2013.txt,2106,"Tue, Jun 25, 2013 at 11:45 AM"
4,10-1-Cascade-2013.txt,2127,"Tue, Jun 25, 2013 at 11:45 AM"


In [23]:
dfs['sent'].head()

,filenm,ln,rest
0,10-1-Cascade-2013.txt,1,"Sunday, July 14, 2013 2:39 PM"
1,10-1-Cascade-2013.txt,20,"Sunday, July 14, 2013 2:39 PM"
2,10-1-Cascade-2013.txt,39,"Monday, July 15, 2013 8:18 AM"
3,10-1-Cascade-2013.txt,192,"Monday, July 15, 2013 8:35 AM"
4,10-1-Cascade-2013.txt,230,"Monday, July 15, 2013 8:35 AM"


In [24]:
for i in range(49):
    print(i, dfs['date'].loc[i, 'rest'])

0 Thursday, July 11, 2013 1:08 PM
1 August 20, 2013, 6:24:26 PM EDT
2 Tue, Jun 25, 2013 at 11:45 AM
3 Tue, Jun 25, 2013 at 11:45 AM
4 Tue, Jun 25, 2013 at 11:45 AM
5 November 19, 2013 at 7:51:38 PM EST
6 November 19, 2013 at 7:51:38 PM EST
7 November 26, 2013 at 11:06:32 AM EST
8 November 26, 2013 at 11:06:32 AM EST
9 November 26, 2013 at 11:06:32 AM EST
10 December 10, 2013 at 4:32:37 PM EST
11 Tue, Dec 10, 2013 2:06 PM
12 December 10, 2013 at 4:32:37 PM EST
13 Tue, Dec 10, 2013 2:06 PM
14 December 10, 2013 at 4:32:37 PM EST
15 Tue, Dec 10, 2013 2:06 PM
16 December 10, 2013 at 4:32:37 PM EST
17 Tue, Dec 10, 2013 2:06 PM
18 December 10, 2013 at 4:32:37 PM EST
19 Tue, Dec 10, 2013 2:06 PM
20 December 10, 2013 at 4:32:37 PM EST
21 Tue, Dec 10, 2013 2:06 PM
22 December 10, 2013 at 4:32:37 PM EST
23 Tue, Dec 10, 2013 2:06 PM
24 December 10, 2013 at 4:32:37 PM EST
25 Tue, Dec 10, 2013 2:06 PM
26 December 10, 2013 at 4:32:37 PM EST
27 Tue, Dec 10, 2013 2:06 PM
28 December 10, 2013 at 4:32:37

In [25]:
dates1 = pd.to_datetime(dfs['date'].loc[:, 'rest'], errors='coerce');
# lines of data_indexes translated to NaT, so are not valid date-times
NaT_date_indexes = dates1[dates1.isna() == True].index

/Users/erlebach/opt/anaconda3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/Users/erlebach/opt/anaconda3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname CST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/Users/erlebach/opt/anaconda3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname PMEDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an excep

In [ ]:
dates1.shape

d = dates.iloc[NaT_date_indexes,:]
for i in range(1000):
    print(d.iloc[i,:])

In [ ]:
sent1 = pd.to_datetime(dfs['sent'].loc[:, 'rest'], exact=False, errors='coerce');
# lines of data_indexes translated to NaT, so are not valid date-times
NaT_sent_indexes = dates1[sent1.isna() == True].index
sent_NaT = dfs['sent'].iloc[NaT_sent_indexes]
NaT_sent_indexes.shape

In [ ]:
# Most errors start with >Sent, which are not original emails, but rather, Re, Fwd, or emails inside threads. 
# So we can remove them. On the other side, 
# sent_NaT.head(50)
count = 0
inputpath="./email_txt/downloaded_from_data_tallahassee/"

rows = []
for row in sent_NaT.iterrows():
    # Only print the lines that do not start with >
    # if sent_NaT.loc[i, 'rest'].value != '>':a
    ln = int(row[1][1])
    fn = row[1][0]
    rest = row[1][2]
    # print("fn: ", fn)
    # print("inputpath: ", inputpath)
    # print("files_dict.keys(): ", files_dict.keys())
    # lines = files_dict[inputpath+fn]
    # print("lines: ", lines[ln : ln+1], "\n")
    # print("ln: ", ln)
    if row[1][2][0] != '>':
        lines = files_dict[inputpath+fn]
        # Took care of zero-based line numbers earlier in this file
        # print("lines: ", lines[ln : ln+2])
        # print(fn, ln, rest)
        # print()
        Lines = " ".join(lines[ln:ln+10])
        # Remove "Sent:" if it occurs at the front
        Lines = re.sub("\b.*?Sent:", "", Lines)
        # Remove leading ">" denoting threads
        Lines = re.sub(">+? Sent:", "", Lines)
        # Remove from To: onward 
        Lines = re.sub("(.*?)(To:.*)", r"\1", Lines)
        Lines = re.sub("\*\*", "", Lines)
        # Fix "...Tuesday August 26:" into "...Tuesdsay August 26, "...
        Lines = re.sub("([a-z] [0-9]*?):", r"\1, ", Lines)
        # Replace ": " by ":" to handle bad time
        Lines = re.sub(": ", ":", Lines)

        # print("row: ", row)
        rows.append(Lines)
        # print("Lines: ", Lines)
        count += 1
        #print("-------------------------")
    #( print(sent_NaT.loc[i, 'rest'])
print("count= ", count)
ddf = pd.DataFrame({'col':rows})
datess = pd.to_datetime(ddf['col'], exact=False, errors='coerce')
print(datess)
print(ddf)
"""
31   Thursday, April 09, 2015 06:04 AM US Mountain...
32   Thursday, April 09, 2015 06:04 AM US Mountain...
Perhaps add a column of time corrections? mostly 0, but +2 hours if US Mountain ...? 
"""
